# Code from Logit.ipynb

#### Imports

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import VotingClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from matplotlib import pyplot

#### Import Data

In [26]:
def get_data():
    url = 'https://raw.githubusercontent.com/COGS118A/Group009-SP23/main/clean_data.csv'
    data = pd.read_csv(url)
    data = data.dropna()
    print(data.shape)
    return data

The provided code performs the following tasks:

1. `get_dataset(data)`: This function takes a `data` input and prepares the dataset for training. It separates the features and the target variable, performs a train-test split with a test size of 0.3 and a random state of 42, and returns the following variables: `X` (features), `y` (target variable), `X_train` (training set features), `X_test` (testing set features), `y_train` (training set target variable), and `y_test` (testing set target variable).

2. `get_models(X_train, X_test, y_train, y_test)`: This function takes the training and testing sets and creates multiple logistic regression models with different hyperparameters. It iterates five times, each time creating a logistic regression model with increasing values for the maximum number of iterations (`max_iter`) and regularization strength (`C`). The models are trained on the `X_train` and `y_train` data, and their predictions are obtained for the `X_test` data. The function returns a dictionary (`models`) where the keys are the model names (e.g., 'Logistic Regression 1', 'Logistic Regression 2', etc.) and the values are dictionaries containing the respective model and its predictions.

3. `get_SV(models, X_train, X_test, y_train, y_test)`: This function takes the models generated in the previous step along with the training and testing sets and creates a soft voting classifier. It extracts the individual models and their names from the `models` dictionary and uses them as estimators for the voting classifier. The voting classifier is then trained on the `X_train` and `y_train` data, and its predictions are obtained for the `X_test` data. The function adds the voting classifier to the `models` dictionary with the key 'Soft Voting' and returns the updated dictionary.

4. `get_accuracy(model, X, y)`: This function calculates the accuracy scores of a given model using cross-validation. It uses a repeated stratified k-fold cross-validation with 10 splits and 3 repeats. The function returns an array of accuracy scores for each fold.

The main part of the code then executes the following steps:

1. Calls `get_dataset(get_data())` to obtain the dataset and assign the returned values to variables `X`, `y`, `X_train`, `X_test`, `y_train`, and `y_test`.

2. Calls `get_models(X_train, X_test, y_train, y_test)` to generate multiple logistic regression models and store them in the `models` dictionary.

3. Calls `get_SV(models, X_train, X_test, y_train, y_test)` to add a soft voting classifier to the `models` dictionary.

4. Initializes empty lists `results` and `names`.

5. Iterates over the items in the `models` dictionary and for each model, performs cross-validation using `get_accuracy(model, X, y)`. The accuracy scores are appended to the `results` list, and the model names are appended to the `names` list. Additionally, the mean and standard deviation of the accuracy scores are printed for each model.

6. Uses `pyplot` to create a boxplot of the accuracy scores (`results`) for each model (`names`). The boxplot is displayed using `pyplot.show()`.

In [ ]:
def get_dataset(data):
    X = data.drop('good_outcome', axis=1)
    y = data['good_outcome']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    return X, y, X_train, X_test, y_train, y_test

def get_models(X_train, X_test, y_train, y_test):
    models = {}

    for i in range(5):
        model_name = f'Logistic Regression {i+1}'  # Generate a unique model name

        model = LogisticRegression(max_iter=(i+1)*1000, C=(i+1)*0.1)

        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        models[model_name] = {
            'model': model,
            'predictions': y_pred
        }
    return models

def get_SV(models,X_train, X_test, y_train, y_test):
    individual_models = []

    for model_name, model_info in models.items():
        individual_models.append((model_name, model_info['model']))

    voting_classifier = VotingClassifier(estimators=individual_models, voting='soft')
    voting_classifier.fit(X_train, y_train)

    y_pred_voting = voting_classifier.predict(X_test)

    models['Soft Voting'] = {
        'model': voting_classifier,
        'predictions': y_pred_voting
    }
    return models

def get_accuracy(model, X, y): #cross valid
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores

X, y, X_train, X_test, y_train, y_test = get_dataset(get_data())

models = get_models(X_train, X_test, y_train, y_test)
models = get_SV(models, X_train, X_test, y_train, y_test)

results, names = list(), list()

for name, model_info in models.items():
    model = model_info['model']
    scores = get_accuracy(model, X, y)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
    
# plot model performance for comparison
pyplot.boxplot(results, labels=[1,2,3,4,5, "Soft Voting"], showmeans=True)
pyplot.show()

(16467, 216)
>Logistic Regression 1 0.845 (0.008)
>Logistic Regression 2 0.845 (0.008)
>Logistic Regression 3 0.845 (0.008)
>Logistic Regression 4 0.844 (0.009)
>Logistic Regression 5 0.844 (0.009)


#### If we had to do it by hand

In [ ]:
import numpy as np

def soft_voting(accuracies):
    weights = accuracies / np.sum(accuracies)
    predictions = np.random.randint(0, 2, size=(len(accuracies), 10))
    ensemble_prediction = np.average(predictions, axis=0, weights=weights)
    
    return np.argmax(ensemble_prediction)